Creating a op property which gives from what operation the current value object is being created.

In [157]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [158]:
class Value:
    def __init__(self,data,_children=(),_op = '',label = ''):
        self.data = data
        self._prev = _children
        self._op = _op
        self.grad = 0.0
        self.label = label
        self._backpropUnit = lambda: None
    
    def __repr__(self):
        return f"Value(data= {self.data})"
    
    def __add__(self,other):
        out =  Value(self.data+other.data,(self,other),'+')
        def _backpropUnit(): # assuming out.grad was set before calling the function here
            self.grad+= 1.0*out.grad
            other.grad+= 1.0*out.grad
        out._backpropUnit = _backpropUnit
        return out
    
    def __mul__(self,other):
        out =  Value(self.data*other.data,(self,other),'*')
        def _backpropUnit(): # assuming out.grad was set before calling the function here
            self.grad +=other.data*out.grad
            other.grad+= self.data*out.grad
        out._backpropUnit = _backpropUnit
        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t,(self,),_op = 'tanh')
        def _backpropUnit(): # assuming out.grad was set before calling the function here
            self.grad += out.grad*(1-t**2)
        out._backpropUnit = _backpropUnit
        return out
    
    def printgrads(self):
        vl = set(self._prev)
        while(len(vl)):
            print([f"{v.label} : {v.grad} " for v in vl])
            nvl = set()
            for v in vl:
                nvl.update(v._prev)
            vl = nvl

    def backprop(self):
        self.clean() # resetting the grads to 0
        self.grad = 1.0
        #applying bfs to calculate grad of each level only after the parent level.
        vl = set()
        vl.add(self)
        while(len(vl)):
            nvl = set()
            for v in vl:
                v._backpropUnit()
                nvl.update(v._prev)
            vl = nvl

    def clean(self):
        nodes = set()
        def _clean(v):
            if v not in nodes:
                nodes.add(v) # ensures atmax only one level deep the repititions will be.
                v.grad = 0
                for child in v._prev:
                    _clean(child)
        _clean(self)



Now here the values are good but in order to be able to perform backpropagation we need a connecting tissue... and it should be such that if c = a+b then c should be able to point to a and b. 
empty tuple as childrens and while initialising we will use set.... but lets for now use list... 

Writing the logic of backpropagation

In [159]:
#inputs x1,x2
x1 = Value(2.0,label='x1')
x2 = Value(-3.0,label='x2')
#weights
w1,w2 = Value(-4.0,label="w1"),Value(5.0,label='w2')
#biases
b = Value(4.5,label='b')
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1+x2w2; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2+b; n.label = 'n'
o = n.tanh(); o.label = 'o'



In [160]:
def backprop(self):
    self.grad = 1.0
    #applying bfs to calculate grad of each level only after the parent level.
    vl = set()
    vl.add(self)
    while(len(vl)):
        nvl = set()
        for v in vl:
            v._backpropUnit()
            nvl.update(v._prev)
        vl = nvl
